# Natural language database description
The BIRD benchmark offers a description of the database (with all the tables and relative columns) in json form. The following function takes that schema from the file ".\Data\dev_tables.json" and creates a textual description of the database in natural language and returns it in string form. The textual description is then saved in the .txt in the file ".\Data\schema_descriptions.txt"

In [2]:
import json

def transform_to_natural_language(schema):
    db_id = schema["db_id"]
    table_names = schema["table_names_original"]
    column_names = schema["column_names"]
    column_types = schema["column_types"]
    primary_keys = schema["primary_keys"]
    foreign_keys = schema["foreign_keys"]

    # Create a dictionary to map table indices to table names
    table_index_to_name = {i: table_names[i] for i in range(len(table_names))}

    # Create a dictionary to map column indices to column names and types
    column_index_to_info = {}
    for idx, (table_idx, col_name) in enumerate(column_names):
        if table_idx == -1:
            continue  # Skip the special "*" column
        table_name = table_index_to_name[table_idx]
        col_type = column_types[idx]
        column_index_to_info[idx] = (table_name, col_name, col_type)

    # Generate the natural language description
    description = f"The database '{db_id}' contains the following tables:\n"

    # Describe each table and its columns
    for table_idx, table_name in table_index_to_name.items():
        description += f"\n- Table name:'{table_name}' has the following columns:\n"
        for col_idx, (tbl_name, col_name, col_type) in column_index_to_info.items():
            if tbl_name == table_name:
                description += f"  - '{col_name}' ({col_type})\n"

    # Describe primary keys
    description += "\nPrimary keys:\n"
    for pk in primary_keys:
        if isinstance(pk, list):
            # Composite primary key
            pk_columns = [column_index_to_info[idx][1] for idx in pk]
            description += f"- Composite primary key on columns: {', '.join(pk_columns)}\n"
        else:
            # Single primary key
            table_name, col_name, _ = column_index_to_info[pk]
            description += f"- Primary key on column '{col_name}' in table '{table_name}'\n"

    # Describe foreign keys
    description += "\nForeign keys:\n"
    for fk in foreign_keys:
        src_col_idx, ref_col_idx = fk
        src_table, src_col, _ = column_index_to_info[src_col_idx]
        ref_table, ref_col, _ = column_index_to_info[ref_col_idx]
        description += f"- Foreign key from column '{src_col}' in table '{src_table}' references column '{ref_col}' in table '{ref_table}'\n"

    return description

In [ ]:
with open(".\Data\dev_tables.json", "r") as file:
    schemas = json.load(file)


with open(".\Data\schema_descriptions.txt", "w") as output_file:
    for schema in schemas:
    # Transform the schema to natural language
            natural_language_description = transform_to_natural_language(schema)
            
            # Write the description to the text file
            output_file.write(natural_language_description)
            output_file.write("\n" + "=" * 80 + "\n")  # Add a separator between schemas

# Extracting table names from SQL queries

the following function parses a SQL query and returns a list of all the talbe retrived from said query

In [4]:
import re

def get_table_from_query(sql_query: str) -> list[str]:
    """
    Function to extract table from SQL query.

    Args:
        query (str): Query to parse

    Returns:
        list: Name of tables taken from query.
    """

    # Regex
    table_pattern = re.compile(r'\bFROM\s+(\w+)(?:\s+AS\s+\w+)?', re.IGNORECASE)
    join_pattern = re.compile(r'\bJOIN\s+(\w+)(?:\s+AS\s+\w+)?', re.IGNORECASE)

    tables = set(table_pattern.findall(sql_query))
    tables.update(join_pattern.findall(sql_query))

    return list(tables)

# Adding STs to the benchmark
for each query of the BIRD benchmark we add in a new column called "STs" all the table names relevant to that query, these will later be used to mesure precision and recall of our system.
the new Dataset is saved in the "./Data/data_with_STs.csv" file

In [ ]:
import pandas as pd
originalData=pd.read_json(".\Data\dev.json")
dataWithSTs=originalData
originalData

,question_id,db_id,question,evidence,SQL,difficulty
0,0,california_schools,What is the highest eligible free rate for K-1...,Eligible free rate for K-12 = `Free Meal Count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,simple
1,1,california_schools,Please list the lowest three eligible free rat...,Eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,moderate
2,2,california_schools,Please list the zip code of all the charter sc...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,simple
3,3,california_schools,What is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,simple
4,4,california_schools,Please list the phone numbers of the direct ch...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,moderate
...,...,...,...,...,...,...
1529,1529,debit_card_specializing,"What is the amount spent by customer ""38508"" a...",January 2012 refers to the Date value = '201201',"SELECT SUM(T1.Price) , SUM(IIF(T3.Date = '2012...",moderate
1530,1530,debit_card_specializing,Which are the top five best selling products? ...,Description of products contains full name,SELECT T2.Description FROM transactions_1k AS ...,simple
1531,1531,debit_card_specializing,Who is the top spending customer and how much ...,average price per single item = Total(price) /...,"SELECT T2.CustomerID, SUM(T2.Price / T2.Amount...",moderate
1532,1532,debit_card_specializing,Which country had the gas station that sold th...,,SELECT T2.Country FROM transactions_1k AS T1 I...,simple


In [ ]:
dataWithSTs["STs"]=dataWithSTs["SQL"].apply(get_table_from_query)

In [ ]:
dataWithSTs

,question_id,db_id,question,evidence,SQL,difficulty,STs
0,0,california_schools,What is the highest eligible free rate for K-1...,Eligible free rate for K-12 = `Free Meal Count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,simple,[frpm]
1,1,california_schools,Please list the lowest three eligible free rat...,Eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,moderate,[frpm]
2,2,california_schools,Please list the zip code of all the charter sc...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,simple,"[frpm, schools]"
3,3,california_schools,What is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,simple,"[frpm, schools]"
4,4,california_schools,Please list the phone numbers of the direct ch...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,moderate,"[frpm, schools]"
...,...,...,...,...,...,...,...
1529,1529,debit_card_specializing,"What is the amount spent by customer ""38508"" a...",January 2012 refers to the Date value = '201201',"SELECT SUM(T1.Price) , SUM(IIF(T3.Date = '2012...",moderate,"[transactions_1k, yearmonth, gasstations]"
1530,1530,debit_card_specializing,Which are the top five best selling products? ...,Description of products contains full name,SELECT T2.Description FROM transactions_1k AS ...,simple,"[transactions_1k, products]"
1531,1531,debit_card_specializing,Who is the top spending customer and how much ...,average price per single item = Total(price) /...,"SELECT T2.CustomerID, SUM(T2.Price / T2.Amount...",moderate,"[transactions_1k, yearmonth, customers]"
1532,1532,debit_card_specializing,Which country had the gas station that sold th...,,SELECT T2.Country FROM transactions_1k AS T1 I...,simple,"[transactions_1k, gasstations]"


In [ ]:
dataWithSTs.to_csv("./Data/data_with_STs.csv")